# Torch Indexing

Master **PyTorch indexing** to access, slice, and modify tensors like a pro ⚡

For each problem, we’ll start with a **naïve Python solution** and the goal is to refactor it using **PyTorch** to make it blazingly fast 🚀 

## 

In [ ]:
import torch
import random
import matplotlib.pyplot as plt
import tqdm

## Problem 1 : Genetic Algorithm

The goal is to learn the weights of a small neural network using a genetic algorithm.

We will simulate **mutation**, **crossover**, and **natural selection** to evolve the network and find the best-performing weights.

In [ ]:
from genetic_env import Simulator, TorchCartPole, TorchPendulum

def mutation_noise_py(pop, mutation_rate=0.1, mutation_scale=0.3):
    """
    Add noise to random weights of the population.
    
    pop : tensor (num_agent, k)
    mutation_rate : chance of adding noise to each weights
    mutation_scale : std of the noise added"""
    (n, k) = pop.shape
    for i in range(n):
        for j in range(k):
            if random.random() < mutation_rate:
                pop[i, j] += random.gauss(0, mutation_scale)
    return pop


def mutation_noise(pop, mutation_rate=0.1, mutation_scale=0.3):
    """
    Add noise to random weights of the population.
    
    pop : tensor (num_agent, k)
    mutation_rate : chance of adding noise to each weights
    mutation_scale : std of the noise added"""
    mask = torch.rand_like(pop) < mutation_rate
    noise = torch.randn_like(pop) * mutation_scale
    pop = pop + mask*noise
    return pop

def crossover_merge(pop, number_new):
    n, k = pop.shape
    # Randomly select pairs of parents
    parents_idx = torch.randint(0, n, (number_new, 2))
    
    parents = pop[parents_idx]
    
    # Create a mask of shape (number_new x k)
    mask = (torch.rand(number_new, k) < 0.5).to(pop.device)
    
    # parent0 genes where mask is True, else parent1 genes
    child = torch.where(mask, parents[:, 0, :], parents[:, 1, :])
    
    return child

def crossover_merge_py(pop, number_new):
    """
    randomly merge weights of elements 2 by 2 of the population
    """
    (n, k) = pop.shape
    pop_new = torch.zeros((number_new, k))
    for l in range(number_new):
        indexes = torch.randint(0, n, (2,))
        for j in range(k):
            if random.random() < 0.5:
                pop_new[l, j] = pop[indexes[0], j]
            else:
                pop_new[l, j] = pop[indexes[1], j]
    return pop_new

def hard_natural_selection_py(pop, rewards, survival_rate=0.5):
    n = pop.shape[0]
    num_survivors = int(n * survival_rate)

    rewards_list = [rewards[i].item() for i in range(n)]

    # Sorted indices of individual of population from best reward to worst reward
    sorted_indices = sorted(range(n), key=lambda i: rewards_list[i], reverse=True)

    # Pick top survivors
    survivors= []
    for i in range(num_survivors):
        survivors.append(pop[sorted_indices[i]])
    return torch.stack(survivors)

def hard_natural_selection(pop, rewards, survival_rate=0.5):
    n = pop.shape[0]
    num_survivors = int(n * survival_rate)
    # Get top survivor indices sorted by descending rewards
    
    _, sorted_indices = torch.sort(rewards, descending=True)
    # Select top survivors directly
    survivors = pop[sorted_indices[:num_survivors]]
    return survivors

def genetic_algorithm(env:Simulator, pop, steps=50, device="cpu", verbose=True, survival_rate=0.1, mutation_rate=0.3, mutation_scale=0.5):
    """ 
    Apply genetic algorithm to optimize the population's reward on the environment
    """
    pop_size = len(pop)
    pop = pop.to(device)
    rewards = env.evaluate(pop).to(device)
    log = []
    enum = range(steps)
    if verbose:
        enum = tqdm.tqdm(enum)
    for _ in enum:
        survivors = hard_natural_selection(pop, rewards, survival_rate=survival_rate)
        best_indiv = survivors[0].unsqueeze(0) # keep the best individual at each step to have only increasing reward
        nomber_of_ind_to_add = pop_size - len(survivors)
        crossed_indviduals = crossover_merge(survivors, nomber_of_ind_to_add).to(device)
        pop = torch.cat([crossed_indviduals, survivors], dim=0).to(device)
        pop = mutation_noise(pop, mutation_rate=mutation_rate, mutation_scale=mutation_scale)
        pop[-1] = best_indiv
        
        rewards = env.evaluate(pop).to(device)
        # logging 
        best_idx = rewards.argmax()
        log.append((rewards[best_idx], pop[best_idx]))
    
    return pop, log
    

device = "cuda" if torch.cuda.is_available() else "cpu"
pop_size = 10
env = TorchPendulum(device)
pop = torch.randn((pop_size, env.weight_size))
pop_optimized, log = genetic_algorithm(env, pop, steps=50, device=device, survival_rate=0.1)
best_indiv = log[-1][1]
env.record_run(best_indiv, video_path="best_pendulum.mp4")
# contains all best individual reward at each optimization step
best_rewards_per_step = [entry[0].item() for entry in log]
plt.plot(best_rewards_per_step)
plt.xlabel('Step')
plt.ylabel('Best Reward')
plt.title('Best Reward over Genetic Algorithm Steps')
plt.grid(True)
plt.show()



## Problem 2 : Particule Swarm Optimization

The goal is to learn the weights of a small neural network using a particule swarm optimization algorithm.

We will simulate **mutation**, **crossover**, and **natural selection** to evolve the network and find the best-performing weights.

In [ ]:
import torch
import tqdm

def particle_swarm_optimization(env, swarm_size=50, steps=100, device="cpu",
                                 w=0.5, phi_p=1.5, phi_g=1.5,
                                 verbose=True):
    """
    Particle Swarm Optimization (PSO) for maximizing environment reward.
    No boundaries — uses Gaussian initialization.
    """

    # Random normal initialization for positions & velocities
    x = torch.randn((swarm_size, env.weight_size), device=device)
    v = torch.randn_like(x)

    # Evaluate initial positions
    rewards = env.evaluate(x)
    p = x.clone()                          # personal best positions
    p_rewards = rewards.clone()            # personal best rewards

    # Global best
    g_idx = p_rewards.argmax()
    g = p[g_idx].clone()
    g_reward = p_rewards[g_idx].clone()

    log = []
    enum = range(steps)
    if verbose:
        enum = tqdm.tqdm(enum)

    for _ in enum:
        # Evaluate current positions
        rewards = env.evaluate(x)

        # Update personal bests
        improved = rewards > p_rewards
        p[improved] = x[improved]
        p_rewards[improved] = rewards[improved]

        # Update global best
        best_idx = p_rewards.argmax()
        if p_rewards[best_idx] > g_reward:
            g = p[best_idx].clone()
            g_reward = p_rewards[best_idx].clone()

        # Update velocities & positions
        rp = torch.rand_like(x)
        rg = torch.rand_like(x)
        v = (w * v
             + phi_p * rp * (p - x)
             + phi_g * rg * (g.unsqueeze(0) - x))
        x = x + v

        # Log best reward so far
        log.append((g_reward.clone(), g.clone()))

    return g, log


device = "cuda" if torch.cuda.is_available() else "cpu"
pop_size = 1000
env = TorchPendulum(device)
pop = torch.randn((pop_size, env.weight_size))
pop_optimized, log = genetic_algorithm(env, pop, steps=50, device=device, survival_rate=0.1)
best_indiv = log[-1][1]
env.record_run(best_indiv, video_path="best_pendulum.mp4")
# contains all best individual reward at each optimization step
best_rewards_per_step = [entry[0].item() for entry in log]
plt.plot(best_rewards_per_step)
plt.xlabel('Step')
plt.ylabel('Best Reward')
plt.title('Best Reward over Genetic Algorithm Steps')
plt.grid(True)
plt.show()


In [ ]:
## Problem : Particule Swarm Optimization

In [ ]:
## Problem : Graph message passing